In [21]:
import pandas as pd
import numpy as np
from numpy import array
import codecs
from os import listdir
import codecs
import re
import string
import pickle
from parsivar import FindStems
from sklearn.linear_model import LogisticRegression
from gensim.models.wrappers import FastText
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.optimizers import SGD
from keras.utils import to_categorical
import spacy
from nltk.stem import PorterStemmer 
language = 'ENGLISH'
path_prefix = ''
dir_prefix = 'saved models/Adjectives/'
if language == 'ENGLISH':
    path_prefix = 'English files/'
    dir_prefix = 'English files/'

In [23]:
print("Let's load...")
if language == 'PERSIAN':
    fastTextModel = FastText.load_fasttext_format('Persian Word Embedding/cc.fa.300')
else:
    fastTextModel = FastText.load_fasttext_format('English Word Embedding/cc.en.300')
print("Loaded !")
# print(model.most_similar('سلام'))

Let's load...
Loaded !


In [24]:
file = None
if language == 'PERSIAN':
    file = open('Persian Word Embedding/cc.fa.300.vec', 'r', encoding='utf-8', errors='ignore')
else:
    file = open('English Word Embedding/cc.en.300.vec', 'r', encoding='utf-8', errors='ignore')

vocab_and_vectors = {}

for line in file:
    values = line.split()
    word = values[0].encode('utf-8').decode('utf-8')
    vector = np.asarray(values[1:], dtype='float32')
    vocab_and_vectors[word] = vector

print(len(vocab_and_vectors))

2000000


In [25]:
print('هیچی' in vocab_and_vectors)
print(dir_prefix)

False
English files/


In [26]:
my_stemmer = None
if language == 'PERSIAN':
    my_stemmer = FindStems()
else:
    my_stemmer = PorterStemmer()
def load_ads(given_path):
    file = open(given_path, 'r', encoding='utf-8')
    text = file.read()
    text = text.split()
    cleaned_text = []
    for each in text:
        if language == 'PERSIAN':
            stemmed_token = my_stemmer.convert_to_stem(each)
        else:
            stemmed_token = my_stemmer.stem(each)
        if stemmed_token in vocab_and_vectors and stemmed_token in fastTextModel:
            cleaned_text.append(stemmed_token)
    file.close()
    return list(set(cleaned_text))

male_adjectives = load_ads(dir_prefix + 'maleAdjectives.txt')
male_adjectives_set = set(male_adjectives)
male_adverbs = load_ads(dir_prefix + 'maleAdverbs.txt')
male_adverbs_set = set(male_adverbs)
female_adjectives = load_ads(dir_prefix + 'femaleAdjectives.txt')
female_adjectives_set = set(female_adjectives)
female_adverbs = load_ads(dir_prefix + 'femaleAdverbs.txt')
female_adverbs_set = set(female_adverbs)
male_verbs = load_ads(dir_prefix + 'maleVerbs.txt')
male_verbs_set = set(male_verbs)
female_verbs = load_ads(dir_prefix + 'femaleVerbs.txt')
female_verbs_set = set(female_verbs)

male_nouns = load_ads(dir_prefix + 'maleNouns.txt')
male_nouns_set = set(male_nouns)
female_nouns = load_ads(dir_prefix + 'femaleNouns.txt')
female_nouns_set = set(female_nouns)
male_ne = load_ads(dir_prefix + 'maleNe.txt')
male_ne_set = set(male_ne)
female_ne = load_ads(dir_prefix + 'femaleNe.txt')
female_ne_set = set(female_ne)
male_conj = load_ads(dir_prefix + 'maleCONJ.txt')
male_conj_set = set(male_conj)
female_conj = load_ads(dir_prefix + 'femaleCONJ.txt')
female_conj_set = set(female_conj)
print(len(male_adjectives), len(female_adjectives), len(male_adverbs), len(female_adverbs))

intersects = set.intersection(male_adjectives_set, male_adverbs_set, female_adjectives_set, female_adverbs_set, male_verbs, female_verbs, male_ne, female_ne, male_nouns, female_nouns, male_conj, female_conj)
male_adjectives_set = male_adjectives_set - intersects
male_adverbs_set = male_adverbs_set - intersects
female_adjectives_set = female_adjectives_set - intersects
female_adverbs_set = female_adverbs_set - intersects
male_verbs_set = male_verbs_set - intersects
female_verbs_set = female_verbs_set - intersects
male_conj_set = male_conj_set - intersects
female_conj_set = female_conj_set - intersects
male_ne_set = male_ne_set - intersects
female_ne_set = female_ne_set - intersects
male_nouns_set = male_nouns_set - intersects
female_nouns_set = female_nouns_set - intersects

mj_mv = set.intersection(male_adjectives_set, male_adverbs_set)
mj_fj = set.intersection(male_adjectives_set, female_adjectives_set)
mj_fv = set.intersection(male_adjectives_set, female_adverbs_set)
mj_mvv = set.intersection(male_adjectives_set, male_verbs_set)
mj_fvv = set.intersection(male_adjectives_set, female_verbs_set)
mj_mne = set.intersection(male_adjectives_set, male_ne_set)
mj_fne = set.intersection(male_adjectives_set, female_ne_set)
mj_mn = set.intersection(male_adjectives_set, male_nouns_set)
mj_fn = set.intersection(male_adjectives_set, female_nouns_set)
mj_mconj = set.intersection(male_adjectives_set, male_conj_set)
mj_fconj = set.intersection(male_adjectives_set, female_conj_set)

mv_fj = set.intersection(male_adverbs_set, female_adjectives_set)
mv_fv = set.intersection(male_adverbs_set, female_adverbs_set)
mv_mvv = set.intersection(male_adverbs_set, male_verbs_set)
mv_fvv = set.intersection(male_adverbs_set, female_verbs_set)
mv_mne = set.intersection(male_adverbs_set, male_ne_set)
mv_fne = set.intersection(male_adverbs_set, female_ne_set)
mv_mn = set.intersection(male_adverbs_set, male_nouns_set)
mv_fn = set.intersection(male_adverbs_set, female_nouns_set)
mv_mconj = set.intersection(male_adverbs_set, male_conj_set)
mv_fconj = set.intersection(male_adverbs_set, female_conj_set)

fj_fv = set.intersection(female_adjectives_set, female_adverbs_set)
fj_mvv = set.intersection(female_adjectives_set, male_verbs_set)
fj_fvv = set.intersection(female_adjectives_set, female_verbs_set)
fj_mne = set.intersection(female_adjectives_set, male_ne_set)
fj_fne = set.intersection(female_adjectives_set, female_ne_set)
fj_mn = set.intersection(female_adjectives_set, male_nouns_set)
fj_fn = set.intersection(female_adjectives_set, female_nouns_set)
fj_mconj = set.intersection(female_adjectives_set, male_conj_set)
fj_fconj = set.intersection(female_adjectives_set, female_conj_set)

fv_mvv = set.intersection(female_adverbs_set, male_verbs_set)
fv_fvv = set.intersection(female_adverbs_set, female_verbs_set)
fv_mne = set.intersection(female_adverbs_set, male_ne_set)
fv_fne = set.intersection(female_adverbs_set, female_ne_set)
fv_mn = set.intersection(female_adverbs_set, male_nouns_set)
fv_fn = set.intersection(female_adverbs_set, female_nouns_set)
fv_mconj = set.intersection(female_adverbs_set, male_conj_set)
fv_fconj = set.intersection(female_adverbs_set, female_conj_set)

mvv_fvv = set.intersection(male_verbs_set, female_verbs_set)
mvv_mne = set.intersection(male_verbs_set, male_ne_set)
mvv_fne = set.intersection(male_verbs_set, female_ne_set)
mvv_mn = set.intersection(male_verbs_set, male_nouns_set)
mvv_fn = set.intersection(male_verbs_set, female_nouns_set)
mvv_mconj = set.intersection(male_verbs_set, male_conj_set)
mvv_fconj = set.intersection(male_verbs_set, female_conj_set)

fvv_mne = set.intersection(female_verbs_set, male_ne_set)
fvv_fne = set.intersection(female_verbs_set, female_ne_set)
fvv_mn = set.intersection(female_verbs_set, male_nouns_set)
fvv_fn = set.intersection(female_verbs_set, female_nouns_set)
fvv_mconj = set.intersection(female_verbs_set, male_conj_set)
fvv_fconj = set.intersection(female_verbs_set, female_conj_set)

mne_fne = set.intersection(male_ne_set, male_ne_set)
mne_mn = set.intersection(male_ne_set, male_nouns_set)
mne_fn = set.intersection(male_ne_set, female_nouns_set)
mne_mconj = set.intersection(male_ne_set, male_conj_set)
mne_fconj = set.intersection(male_ne_set, female_conj_set)

fne_mn = set.intersection(female_ne_set, male_nouns_set)
fne_fn = set.intersection(female_ne_set, female_nouns_set)
fne_mconj = set.intersection(female_ne_set, male_conj_set)
fne_fconj = set.intersection(female_ne_set, female_conj_set)

mn_fn = set.intersection(male_nouns_set, female_nouns_set)
mn_mconj = set.intersection(male_nouns_set, male_conj_set)
mn_fconj = set.intersection(male_nouns_set, female_conj_set)

fn_mconj = set.intersection(female_nouns_set, male_conj_set)
fn_fconj = set.intersection(female_nouns_set, female_conj_set)

mconj_fconj = set.intersection(male_conj_set, female_conj_set)

male_adjectives_set = male_adjectives_set - mj_mv
male_adjectives_set = male_adjectives_set - mj_fj
male_adjectives_set = male_adjectives_set - mj_fv
male_adjectives_set = male_adjectives_set - mj_mvv
male_adjectives_set = male_adjectives_set - mj_fvv
male_adjectives_set = male_adjectives_set - mj_mn
male_adjectives_set = male_adjectives_set - mj_fn
male_adjectives_set = male_adjectives_set - mj_mne
male_adjectives_set = male_adjectives_set - mj_fne
male_adjectives_set = male_adjectives_set - mj_mconj
male_adjectives_set = male_adjectives_set - mj_fconj
male_adverbs_set = male_adverbs_set - mj_mv
male_adverbs_set = male_adverbs_set - mv_fj
male_adverbs_set = male_adverbs_set - mv_fv
male_adverbs_set = male_adverbs_set - mv_mvv
male_adverbs_set = male_adverbs_set - mv_fvv
male_adverbs_set = male_adverbs_set - mv_mn
male_adverbs_set = male_adverbs_set - mv_fn
male_adverbs_set = male_adverbs_set - mv_mne
male_adverbs_set = male_adverbs_set - mv_fne
male_adverbs_set = male_adverbs_set - mv_mconj
male_adverbs_set = male_adverbs_set - mv_fconj
female_adjectives_set = female_adjectives_set - mj_fj
female_adjectives_set = female_adjectives_set - mv_fj
female_adjectives_set = female_adjectives_set - fj_fv
female_adjectives_set = female_adjectives_set - fj_mvv
female_adjectives_set = female_adjectives_set - fj_fvv
female_adjectives_set = female_adjectives_set - fj_mn
female_adjectives_set = female_adjectives_set - fj_fn
female_adjectives_set = female_adjectives_set - fj_mne
female_adjectives_set = female_adjectives_set - fj_fne
female_adjectives_set = female_adjectives_set - fj_mconj
female_adjectives_set = female_adjectives_set - fj_fconj
female_adverbs_set = female_adverbs_set - mj_fv
female_adverbs_set = female_adverbs_set - mv_fv
female_adverbs_set = female_adverbs_set - fj_fv
female_adverbs_set = female_adverbs_set - fv_mvv
female_adverbs_set = female_adverbs_set - fv_fvv
female_adverbs_set = female_adverbs_set - fv_mn
female_adverbs_set = female_adverbs_set - fv_fn
female_adverbs_set = female_adverbs_set - fv_mne
female_adverbs_set = female_adverbs_set - fv_fne
female_adverbs_set = female_adverbs_set - fv_mconj
female_adverbs_set = female_adverbs_set - fv_fconj
male_verbs_set = male_verbs_set - mj_mvv
male_verbs_set = male_verbs_set - mv_mvv
male_verbs_set = male_verbs_set - fj_mvv
male_verbs_set = male_verbs_set - fv_mvv
male_verbs_set = male_verbs_set - mvv_fvv
male_verbs_set = male_verbs_set - mvv_mn
male_verbs_set = male_verbs_set - mvv_fn
male_verbs_set = male_verbs_set - mvv_mne
male_verbs_set = male_verbs_set - mvv_fne
male_verbs_set = male_verbs_set - mvv_mconj
male_verbs_set = male_verbs_set - mvv_fconj
female_verbs_set = female_verbs_set - mj_fvv
female_verbs_set = female_verbs_set - mv_fvv
female_verbs_set = female_verbs_set - fj_fvv
female_verbs_set = female_verbs_set - fv_fvv
female_verbs_set = female_verbs_set - mvv_fvv
female_verbs_set = female_verbs_set - fvv_mn
female_verbs_set = female_verbs_set - fvv_fn
female_verbs_set = female_verbs_set - fvv_mne
female_verbs_set = female_verbs_set - fvv_fne
female_verbs_set = female_verbs_set - fvv_mconj
female_verbs_set = female_verbs_set - fvv_fconj
male_nouns_set = male_nouns_set - mj_mn
male_nouns_set = male_nouns_set - mv_mn
male_nouns_set = male_nouns_set - fj_mn
male_nouns_set = male_nouns_set - fv_mn
male_nouns_set = male_nouns_set - mvv_mn
male_nouns_set = male_nouns_set - fvv_mn
male_nouns_set = male_nouns_set - mn_fn
male_nouns_set = male_nouns_set - mne_mn
male_nouns_set = male_nouns_set - fne_mn
male_nouns_set = male_nouns_set - mn_mconj
male_nouns_set = male_nouns_set - mn_fconj
female_nouns_set = female_nouns_set - mj_fn
female_nouns_set = female_nouns_set - mv_fn
female_nouns_set = female_nouns_set - fj_fn
female_nouns_set = female_nouns_set - fv_fn
female_nouns_set = female_nouns_set - mvv_fn
female_nouns_set = female_nouns_set - fvv_fn
female_nouns_set = female_nouns_set - mn_fn
female_nouns_set = female_nouns_set - mne_fn
female_nouns_set = female_nouns_set - fne_fn
female_nouns_set = female_nouns_set - fn_mconj
female_nouns_set = female_nouns_set - fn_fconj
male_ne_set = male_ne_set - mj_mne
male_ne_set = male_ne_set - mv_mne
male_ne_set = male_ne_set - fj_mne
male_ne_set = male_ne_set - fv_mne
male_ne_set = male_ne_set - mvv_mne
male_ne_set = male_ne_set - fvv_mne
male_ne_set = male_ne_set - mne_mn
male_ne_set = male_ne_set - mne_fn
male_ne_set = male_ne_set - mne_fne
male_ne_set = male_ne_set - mne_mconj
male_ne_set = male_ne_set - mne_fconj
female_ne_set = female_ne_set - mj_fne
female_ne_set = female_ne_set - mv_fne
female_ne_set = female_ne_set - fj_fne
female_ne_set = female_ne_set - fv_fne
female_ne_set = female_ne_set - mvv_fne
female_ne_set = female_ne_set - fvv_fne
female_ne_set = female_ne_set - fne_mn
female_ne_set = female_ne_set - fne_fn
female_ne_set = female_ne_set - mne_fne
female_ne_set = female_ne_set - fne_mconj
female_ne_set = female_ne_set - fne_fconj
male_conj_set = male_conj_set - mj_mconj
male_conj_set = male_conj_set - mv_mconj
male_conj_set = male_conj_set - fj_mconj
male_conj_set = male_conj_set - fv_mconj
male_conj_set = male_conj_set - mvv_mconj
male_conj_set = male_conj_set - fvv_mconj
male_conj_set = male_conj_set - mn_mconj
male_conj_set = male_conj_set - fn_mconj
male_conj_set = male_conj_set - mne_mconj
male_conj_set = male_conj_set - fne_mconj
male_conj_set = male_conj_set - mconj_fconj
female_conj_set = female_conj_set - mj_fconj
female_conj_set = female_conj_set - mv_fconj
female_conj_set = female_conj_set - fj_fconj
female_conj_set = female_conj_set - fv_fconj
female_conj_set = female_conj_set - mvv_fconj
female_conj_set = female_conj_set - fvv_fconj
female_conj_set = female_conj_set - mn_fconj
female_conj_set = female_conj_set - fn_fconj
female_conj_set = female_conj_set - mne_fconj
female_conj_set = female_conj_set - fne_fconj
female_conj_set = female_conj_set - mconj_fconj

male_adjectives = list(male_adjectives_set)
male_adverbs = list(male_adverbs_set)
female_adjectives = list(female_adjectives_set)
female_adverbs = list(female_adverbs_set)
male_verbs = list(male_verbs_set)
female_verbs = list(female_verbs_set)
male_nouns = list(male_nouns_set)
female_nouns = list(female_nouns_set)
male_ne = list(male_ne_set)
female_ne = list(female_ne_set)
male_conj = list(male_conj_set)
female_conj = list(female_conj_set)
vocabulary = set(male_adjectives + male_adverbs + female_adjectives + female_adverbs + male_verbs + female_verbs + male_nouns + female_nouns + male_ne + female_ne + male_conj + female_conj)
    
all_data = male_adjectives + male_adverbs + male_verbs + male_nouns + male_ne + male_conj + female_adjectives + female_adverbs + female_verbs + female_nouns + female_ne + female_conj
all_labels = [1] * (len(male_adjectives) + len(male_adverbs) + len(male_verbs) + len(male_nouns) + len(male_ne) + len(male_conj)) + [0] * (len(female_adjectives) + len(female_adverbs) + len(female_verbs) + len(female_nouns) + len(female_ne) + len(female_conj))
print("Intersection:",len(set.intersection(set(male_adjectives), set(male_adverbs), set(female_adjectives), set(female_adverbs), set(male_verbs), set(female_verbs))))
print(len(all_data), len(all_labels))

2267 2179 574 576
Intersection: 0
2740 2740


In [27]:
print(all_data[0])
print(len(all_labels), sum(all_labels))
print(len(male_adjectives))

liber
2740 2102
126


In [29]:
X = [list(each) for each in all_data if len(each) > 1]
english_chars = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
chars = None
if language == 'PERSIAN':
    chars = ['الف', 'ب', 'پ', 'ت', 'ث', 'ج', 'چ', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'ژ', 'س' ,'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ک', 'گ', 'ل', 'م', 'ن', 'و', 'ه', 'ی']
else:
    chars = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
chars = set(chars)
for eachData in all_data:
    for eachChar in eachData:
        if language == 'PERSIAN' and (eachChar not in english_chars):
            chars.add(eachChar)
        if language == 'ENGLISH':
            chars.add(eachChar)
        
print(len(list(chars)))
chars = sorted(list(chars))
mapping = dict((c, i) for i, c in enumerate(chars))
sequences = []
for each in all_data:
    encoded_sequence = [mapping[eachOne] for eachOne in each if eachOne in mapping]
    sequences.append(encoded_sequence)
    
vocab_size = len(mapping)
sequences = np.array(sequences)
sequences = [to_categorical(x, num_classes=vocab_size) for x in sequences]

# max_length = vocab_size
X = pad_sequences(sequences, padding='post')
y = all_labels

print(np.array(X).shape, np.array(y).shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True)
max_length = max([each.shape[0] for each in X_train])
print('max_length', max_length)

52
(2740, 13, 52) (2740,)
max_length 13


In [30]:
print(vocab_size, max_length)
print(X.shape[1], X.shape[2])

52 13
13 52


In [11]:
def define_model(vocab_size, max_length):
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=8, activation='relu', input_shape=(X.shape[1], X.shape[2])))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(125))
    model.add(Dense(vocab_size, activation='relu'))
    model.add(Dense(1, activation='sigmoid' ))
    model. compile(loss='binary_crossentropy' , optimizer='adam' , metrics=['accuracy'])
    model.summary()
    return model

model = define_model(vocab_size, max_length)
checkpoint = ModelCheckpoint(path_prefix + "object.h5", monitor='val_acc', mode='max')
model.fit(X_train, y_train, epochs=200, validation_data = (X_val, y_val), batch_size=256, callbacks=[checkpoint])

if language == 'PERSIAN':
    model.save('saved models/Keras/object.h5' )
else:
    model.save(path_prefix + 'objectMain.h5' )







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 8, 32)             11040     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 4, 32)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 125)               79000     
_________________________________________________________________
dense_1 (Dense)              (None, 43)                5418      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 44        
Total params: 95,502
Trainable params: 95,502
Non-trainable params: 0
_________________________________________________________________



Train on 15466 samples, validate on

15466/15466 [==============================] - ETA: 1s - loss: 0.5950 - acc: 0.648 - ETA: 2s - loss: 0.5644 - acc: 0.682 - ETA: 1s - loss: 0.5500 - acc: 0.703 - ETA: 1s - loss: 0.5542 - acc: 0.700 - ETA: 1s - loss: 0.5615 - acc: 0.694 - ETA: 1s - loss: 0.5677 - acc: 0.686 - ETA: 1s - loss: 0.5657 - acc: 0.693 - ETA: 1s - loss: 0.5705 - acc: 0.688 - ETA: 1s - loss: 0.5704 - acc: 0.690 - ETA: 1s - loss: 0.5724 - acc: 0.688 - ETA: 1s - loss: 0.5710 - acc: 0.691 - ETA: 1s - loss: 0.5691 - acc: 0.694 - ETA: 1s - loss: 0.5665 - acc: 0.696 - ETA: 1s - loss: 0.5650 - acc: 0.698 - ETA: 1s - loss: 0.5639 - acc: 0.699 - ETA: 0s - loss: 0.5658 - acc: 0.698 - ETA: 0s - loss: 0.5642 - acc: 0.700 - ETA: 0s - loss: 0.5638 - acc: 0.701 - ETA: 0s - loss: 0.5642 - acc: 0.701 - ETA: 0s - loss: 0.5647 - acc: 0.701 - ETA: 0s - loss: 0.5661 - acc: 0.700 - ETA: 0s - loss: 0.5659 - acc: 0.701 - ETA: 0s - loss: 0.5653 - acc: 0.701 - ETA: 0s - loss: 0.5681 - acc: 0.698 - ETA: 0s - loss: 0.5697 - acc: 0.696 - ETA

15466/15466 [==============================] - ETA: 2s - loss: 0.4506 - acc: 0.769 - ETA: 1s - loss: 0.4231 - acc: 0.800 - ETA: 2s - loss: 0.4141 - acc: 0.808 - ETA: 2s - loss: 0.4175 - acc: 0.807 - ETA: 2s - loss: 0.4220 - acc: 0.807 - ETA: 2s - loss: 0.4223 - acc: 0.808 - ETA: 2s - loss: 0.4214 - acc: 0.807 - ETA: 2s - loss: 0.4314 - acc: 0.802 - ETA: 2s - loss: 0.4252 - acc: 0.803 - ETA: 2s - loss: 0.4275 - acc: 0.801 - ETA: 2s - loss: 0.4298 - acc: 0.801 - ETA: 2s - loss: 0.4282 - acc: 0.801 - ETA: 2s - loss: 0.4296 - acc: 0.800 - ETA: 1s - loss: 0.4266 - acc: 0.802 - ETA: 1s - loss: 0.4252 - acc: 0.801 - ETA: 1s - loss: 0.4260 - acc: 0.800 - ETA: 1s - loss: 0.4245 - acc: 0.801 - ETA: 1s - loss: 0.4269 - acc: 0.800 - ETA: 1s - loss: 0.4259 - acc: 0.800 - ETA: 1s - loss: 0.4269 - acc: 0.799 - ETA: 1s - loss: 0.4277 - acc: 0.798 - ETA: 0s - loss: 0.4275 - acc: 0.797 - ETA: 0s - loss: 0.4287 - acc: 0.795 - ETA: 0s - loss: 0.4325 - acc: 0.792 - ETA: 0s - loss: 0.4324 - acc: 0.792 - ETA

Epoch 26/200
15466/15466 [==============================] - ETA: 2s - loss: 0.4144 - acc: 0.800 - ETA: 2s - loss: 0.3833 - acc: 0.826 - ETA: 1s - loss: 0.3869 - acc: 0.828 - ETA: 1s - loss: 0.3938 - acc: 0.823 - ETA: 1s - loss: 0.3944 - acc: 0.823 - ETA: 1s - loss: 0.3906 - acc: 0.824 - ETA: 1s - loss: 0.3904 - acc: 0.822 - ETA: 1s - loss: 0.3877 - acc: 0.823 - ETA: 1s - loss: 0.3866 - acc: 0.824 - ETA: 1s - loss: 0.3928 - acc: 0.821 - ETA: 1s - loss: 0.3893 - acc: 0.823 - ETA: 1s - loss: 0.3911 - acc: 0.822 - ETA: 1s - loss: 0.3920 - acc: 0.820 - ETA: 1s - loss: 0.3931 - acc: 0.818 - ETA: 1s - loss: 0.3990 - acc: 0.813 - ETA: 1s - loss: 0.4009 - acc: 0.811 - ETA: 1s - loss: 0.4025 - acc: 0.810 - ETA: 1s - loss: 0.4037 - acc: 0.809 - ETA: 1s - loss: 0.4062 - acc: 0.807 - ETA: 0s - loss: 0.4078 - acc: 0.806 - ETA: 0s - loss: 0.4085 - acc: 0.806 - ETA: 0s - loss: 0.4101 - acc: 0.804 - ETA: 0s - loss: 0.4112 - acc: 0.802 - ETA: 0s - loss: 0.4124 - acc: 0.801 - ETA: 0s - loss: 0.4124 - acc

15466/15466 [==============================] - ETA: 1s - loss: 0.3017 - acc: 0.863 - ETA: 1s - loss: 0.3298 - acc: 0.850 - ETA: 1s - loss: 0.3339 - acc: 0.851 - ETA: 1s - loss: 0.3337 - acc: 0.851 - ETA: 1s - loss: 0.3330 - acc: 0.852 - ETA: 1s - loss: 0.3349 - acc: 0.854 - ETA: 1s - loss: 0.3388 - acc: 0.851 - ETA: 1s - loss: 0.3392 - acc: 0.852 - ETA: 1s - loss: 0.3365 - acc: 0.855 - ETA: 1s - loss: 0.3379 - acc: 0.855 - ETA: 1s - loss: 0.3398 - acc: 0.853 - ETA: 0s - loss: 0.3412 - acc: 0.851 - ETA: 0s - loss: 0.3428 - acc: 0.850 - ETA: 0s - loss: 0.3434 - acc: 0.849 - ETA: 0s - loss: 0.3420 - acc: 0.849 - ETA: 0s - loss: 0.3434 - acc: 0.848 - ETA: 0s - loss: 0.3427 - acc: 0.848 - ETA: 0s - loss: 0.3447 - acc: 0.848 - ETA: 0s - loss: 0.3459 - acc: 0.847 - ETA: 0s - loss: 0.3470 - acc: 0.846 - ETA: 0s - loss: 0.3478 - acc: 0.845 - ETA: 0s - loss: 0.3493 - acc: 0.843 - ETA: 0s - loss: 0.3514 - acc: 0.842 - ETA: 0s - loss: 0.3513 - acc: 0.843 - ETA: 0s - loss: 0.3518 - acc: 0.843 - ETA

...

15466/15466 [==============================] - ETA: 2s - loss: 0.1135 - acc: 0.960 - ETA: 1s - loss: 0.1157 - acc: 0.950 - ETA: 1s - loss: 0.1149 - acc: 0.951 - ETA: 1s - loss: 0.1302 - acc: 0.945 - ETA: 1s - loss: 0.1305 - acc: 0.945 - ETA: 1s - loss: 0.1318 - acc: 0.945 - ETA: 1s - loss: 0.1286 - acc: 0.947 - ETA: 1s - loss: 0.1378 - acc: 0.942 - ETA: 1s - loss: 0.1408 - acc: 0.940 - ETA: 1s - loss: 0.1405 - acc: 0.941 - ETA: 1s - loss: 0.1416 - acc: 0.940 - ETA: 1s - loss: 0.1405 - acc: 0.941 - ETA: 1s - loss: 0.1386 - acc: 0.942 - ETA: 1s - loss: 0.1365 - acc: 0.943 - ETA: 1s - loss: 0.1355 - acc: 0.943 - ETA: 1s - loss: 0.1354 - acc: 0.943 - ETA: 1s - loss: 0.1362 - acc: 0.943 - ETA: 1s - loss: 0.1399 - acc: 0.942 - ETA: 1s - loss: 0.1391 - acc: 0.942 - ETA: 1s - loss: 0.1398 - acc: 0.942 - ETA: 0s - loss: 0.1414 - acc: 0.941 - ETA: 0s - loss: 0.1419 - acc: 0.941 - ETA: 0s - loss: 0.1411 - acc: 0.942 - ETA: 0s - loss: 0.1415 - acc: 0.942 - ETA: 0s - loss: 0.1406 - acc: 0.942 - ETA

15466/15466 [==============================] - ETA: 2s - loss: 0.1042 - acc: 0.960 - ETA: 2s - loss: 0.1168 - acc: 0.954 - ETA: 2s - loss: 0.1333 - acc: 0.946 - ETA: 1s - loss: 0.1431 - acc: 0.943 - ETA: 1s - loss: 0.1426 - acc: 0.942 - ETA: 1s - loss: 0.1548 - acc: 0.938 - ETA: 1s - loss: 0.1631 - acc: 0.935 - ETA: 1s - loss: 0.1601 - acc: 0.937 - ETA: 1s - loss: 0.1605 - acc: 0.936 - ETA: 1s - loss: 0.1602 - acc: 0.936 - ETA: 1s - loss: 0.1634 - acc: 0.934 - ETA: 1s - loss: 0.1603 - acc: 0.935 - ETA: 1s - loss: 0.1579 - acc: 0.935 - ETA: 1s - loss: 0.1581 - acc: 0.935 - ETA: 1s - loss: 0.1597 - acc: 0.935 - ETA: 0s - loss: 0.1627 - acc: 0.934 - ETA: 0s - loss: 0.1628 - acc: 0.934 - ETA: 0s - loss: 0.1672 - acc: 0.932 - ETA: 0s - loss: 0.1690 - acc: 0.931 - ETA: 0s - loss: 0.1689 - acc: 0.931 - ETA: 0s - loss: 0.1701 - acc: 0.929 - ETA: 0s - loss: 0.1679 - acc: 0.930 - ETA: 0s - loss: 0.1702 - acc: 0.930 - ETA: 0s - loss: 0.1695 - acc: 0.930 - ETA: 0s - loss: 0.1685 - acc: 0.930 - ETA

15466/15466 [==============================] - ETA: 2s - loss: 0.1647 - acc: 0.949 - ETA: 2s - loss: 0.1224 - acc: 0.958 - ETA: 1s - loss: 0.1151 - acc: 0.959 - ETA: 1s - loss: 0.1099 - acc: 0.960 - ETA: 1s - loss: 0.1027 - acc: 0.964 - ETA: 1s - loss: 0.1038 - acc: 0.964 - ETA: 1s - loss: 0.1020 - acc: 0.965 - ETA: 1s - loss: 0.1018 - acc: 0.965 - ETA: 1s - loss: 0.0998 - acc: 0.966 - ETA: 1s - loss: 0.1007 - acc: 0.964 - ETA: 1s - loss: 0.1009 - acc: 0.965 - ETA: 1s - loss: 0.0994 - acc: 0.965 - ETA: 1s - loss: 0.1000 - acc: 0.965 - ETA: 1s - loss: 0.1011 - acc: 0.964 - ETA: 1s - loss: 0.1013 - acc: 0.964 - ETA: 0s - loss: 0.1022 - acc: 0.963 - ETA: 0s - loss: 0.1025 - acc: 0.963 - ETA: 0s - loss: 0.1015 - acc: 0.963 - ETA: 0s - loss: 0.1016 - acc: 0.963 - ETA: 0s - loss: 0.1019 - acc: 0.963 - ETA: 0s - loss: 0.1013 - acc: 0.963 - ETA: 0s - loss: 0.1025 - acc: 0.962 - ETA: 0s - loss: 0.1022 - acc: 0.962 - ETA: 0s - loss: 0.1024 - acc: 0.962 - ETA: 0s - loss: 0.1018 - acc: 0.962 - ETA

15466/15466 [==============================] - ETA: 2s - loss: 0.2046 - acc: 0.914 - ETA: 1s - loss: 0.1533 - acc: 0.934 - ETA: 1s - loss: 0.1512 - acc: 0.935 - ETA: 1s - loss: 0.1312 - acc: 0.945 - ETA: 1s - loss: 0.1350 - acc: 0.942 - ETA: 1s - loss: 0.1421 - acc: 0.938 - ETA: 1s - loss: 0.1397 - acc: 0.939 - ETA: 1s - loss: 0.1381 - acc: 0.941 - ETA: 1s - loss: 0.1361 - acc: 0.944 - ETA: 1s - loss: 0.1394 - acc: 0.943 - ETA: 1s - loss: 0.1365 - acc: 0.944 - ETA: 1s - loss: 0.1386 - acc: 0.943 - ETA: 1s - loss: 0.1397 - acc: 0.942 - ETA: 1s - loss: 0.1379 - acc: 0.943 - ETA: 1s - loss: 0.1376 - acc: 0.944 - ETA: 0s - loss: 0.1401 - acc: 0.943 - ETA: 0s - loss: 0.1408 - acc: 0.943 - ETA: 0s - loss: 0.1397 - acc: 0.943 - ETA: 0s - loss: 0.1388 - acc: 0.944 - ETA: 0s - loss: 0.1379 - acc: 0.944 - ETA: 0s - loss: 0.1378 - acc: 0.944 - ETA: 0s - loss: 0.1374 - acc: 0.944 - ETA: 0s - loss: 0.1393 - acc: 0.943 - ETA: 0s - loss: 0.1389 - acc: 0.943 - ETA: 0s - loss: 0.1376 - acc: 0.944 - ETA

15466/15466 [==============================] - ETA: 1s - loss: 0.0947 - acc: 0.968 - ETA: 1s - loss: 0.0974 - acc: 0.964 - ETA: 1s - loss: 0.1107 - acc: 0.955 - ETA: 1s - loss: 0.1088 - acc: 0.956 - ETA: 1s - loss: 0.1117 - acc: 0.953 - ETA: 1s - loss: 0.1126 - acc: 0.952 - ETA: 1s - loss: 0.1114 - acc: 0.951 - ETA: 1s - loss: 0.1079 - acc: 0.953 - ETA: 1s - loss: 0.1076 - acc: 0.953 - ETA: 1s - loss: 0.1104 - acc: 0.952 - ETA: 1s - loss: 0.1123 - acc: 0.952 - ETA: 1s - loss: 0.1107 - acc: 0.953 - ETA: 1s - loss: 0.1109 - acc: 0.954 - ETA: 1s - loss: 0.1107 - acc: 0.954 - ETA: 1s - loss: 0.1113 - acc: 0.953 - ETA: 0s - loss: 0.1102 - acc: 0.954 - ETA: 0s - loss: 0.1093 - acc: 0.954 - ETA: 0s - loss: 0.1094 - acc: 0.955 - ETA: 0s - loss: 0.1093 - acc: 0.955 - ETA: 0s - loss: 0.1082 - acc: 0.955 - ETA: 0s - loss: 0.1088 - acc: 0.955 - ETA: 0s - loss: 0.1080 - acc: 0.956 - ETA: 0s - loss: 0.1071 - acc: 0.956 - ETA: 0s - loss: 0.1072 - acc: 0.956 - ETA: 0s - loss: 0.1070 - acc: 0.957 - ETA

In [12]:
model = None
if language == 'PERSIAN':
    model = load_model('saved models/Keras/object.h5' )
else:
    model = load_model(path_prefix + 'objectMain.h5' )

print('done')
_, acc = model.evaluate(X_train, y_train)
print('Train Accuracy: %.2f' % (acc*100))
_, acc = model.evaluate(X_test, y_test)
print('Test Accuracy: %.2f' % (acc*100))

done
15466/15466 [==============================] - ETA: 3: - ETA: 33s - ETA: 15 - ETA: 10 - ETA: 8 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 3s 211us/step
Train Accuracy: 95.88
4834/4834 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 190us/step
Test Accuracy: 54.36


In [13]:
with open(path_prefix + 'CharDict.pickle', 'wb') as f:
    pickle.dump(mapping, f, pickle.HIGHEST_PROTOCOL)